In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [4]:
lambdas = np.array([8,2])/0.8

In [ ]:
folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250611 LSU eps 9"
project_name = "20250611 different ff"
postprocess_results = []
runtime_ps = 25e-12
min_steps_per_lambda = 20
cuts = [1]
h5_bg = None
ref = True
run = False
for direction in ["z"]: 
    for f,filename in enumerate(os.listdir(folder_path)):
     print(filename)
     for cut in cuts:
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            a=get_a_from_h5_eps(file,14.3*0.8)
            if run:
                structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                                ,direction=direction, lambda_range=lambdas,
                                                box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                               scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                               monitors=["flux"], flux_monitor_position=18,cell_size_manual=45,
                                               freqs=500, 
                                               cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                                               )

                file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value}\z_incidence\{structure_1.sim_name}.txt"
                if os.path.exists(file_desc):
                    print("Exist!")
                else:
                   print("Creating...")
                   structure_1.run_sim(run_free=False,load=False,add_ref=ref,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value:.2f}",monitor=True)
                   if ref:
                      ref = False

        # del structure_1

        # structure_1.plot_sim_layout()
        # structure_1.estimate_cost()
        # raise(Exception)

